In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LassoCV
import ast
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandas as pd

from matplotlib.colors import Normalize, LinearSegmentedColormap
from matplotlib.cm import ScalarMappable
import matplotlib.colors as mcolors
family_to_color = {
    'IL-1': '#2D2D2A',
    'IL-10': '#4C4C47',
    'IL-17': '#686E76',
    'IL-6/IL-12': '#848FA5',
    'IL-10 (IL-20 subfamily)': '#B9BFCB',
    'Interleukin - non-classified': '#DCDFE5',
    'Interferon': '#947E91',
    'Common γ chain/IL-13/TSLP': '#C14953',
    'TNF': '#D3938C',
    'Growth factor': '#E1C3B7',
    'Other': '#E5DCC5',
    
    'Common β chain': '#B76F52',
    'Chemokine': '#D0B4BC',
    'Complement': '#7F6B39'
}

/home/icb/dominik.klein/mambaforge/envs/cfp/lib/python3.11/site-packages/optuna/study/_optimize.py:29: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/figures/pbmc/data/donor_feature_importance_per_cell_type"
data_output_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/feature_importance"

In [3]:
data_dir = "/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/metrics_new_donor"
df = pd.read_csv(os.path.join(data_dir, "metrics_new_donor.csv"), index_col=0)

In [4]:

with open("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/cyto_order.pkl", "rb") as file:
    cyto_order = pickle.load(file)

with open("/lustre/groups/ml01/workspace/ot_perturbation/data/pbmc/cyto_to_fam.pkl", "rb") as file:
    cyto_to_family = pickle.load(file)


In [5]:
df.head()

,mean_decoded_r_sq_per_cell_type,mean_e_distance_per_cell_type,mean_mmd_per_cell_type,mean_sdiv_10_per_cell_type,mean_sdiv_100_per_cell_type,mean_deg_r_sq_per_cell_type,mean_deg_e_distance_per_cell_type,mean_deg_mmd_per_cell_type,mean_deg_sdiv_10_per_cell_type,mean_deg_sdiv_100_per_cell_type,...,e_distance_pDC,mmd_pDC,div_10_pDC,div_100_pDC,deg_decoded_r_squared_pDC,deg_e_distance_pDC,deg_mmd_pDC,deg_div_10_pDC,deg_div_100_pDC,new_run
Donor8_IL-1Ra,0.936079,18.672976,0.010558,99.468785,15.969938,0.835870,27.137565,0.039739,137.779715,22.365724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor12_IL-32-beta,0.880287,57.002564,0.019194,131.728360,43.893835,0.488720,78.095042,0.063817,178.566758,52.956970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor8_FasL,0.936261,20.548462,0.010757,101.143822,17.251914,0.816263,26.480086,0.042731,139.250231,22.540262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor7_OX40L,0.923112,29.121615,0.011908,104.739377,22.172538,0.763669,36.172174,0.051545,146.038442,29.380676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor5_BAFF,0.920968,30.491248,0.012784,105.387378,23.400425,0.689538,35.676550,0.049371,156.660900,28.123338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [6]:
def get_cytokine_importance(all_cytokines_in_train: list[str], df_d: pd.DataFrame, cytokine: str, target_variable: str = "ood_e_distance"):
    
    for d in all_cytokines_in_train:
        df_d[f"{d}_present"] = df_d.apply(lambda x: d in x["cytokine_in_train"], axis=1)

    for num_cytokines in df["num_cytokines_in_train"].unique():
        df_d[f"{num_cytokines}_present"] = df_d.apply(lambda x: x["num_cytokines_in_train"] == num_cytokines, axis=1)
    
    cols = [f"{d}_present" for d in all_cytokines_in_train] + [f"{num_cytokines}_present" for num_cytokines in df_d["num_cytokines_in_train"].unique()]
    # Prepare feature matrix and target variable
    X = df_d[cols]
    y = df_d[target_variable]
    
    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Fit Lasso regression
    lasso = LassoCV(max_iter=5000, cv=5, random_state=0)
    lasso.fit(X_scaled, y)
    
    # Extract coefficients
    coefficients = lasso.coef_
    intercept = lasso.intercept_
    
    # Create a DataFrame with feature importance
    feature_stats = pd.DataFrame({
        'Feature': cols,
        'Coefficient': coefficients
    }).sort_values(by='Coefficient', ascending=False)
    
    # Add the intercept for completeness
    feature_stats.loc[len(feature_stats)] = ['Intercept', intercept]
    
    return feature_stats



In [7]:
all_cytokines_in_test = df["cytokine"].unique()

all_cytokines_in_train = list(
        set(el for i in range(len(df)) for el in ast.literal_eval(df.iloc[i]["cytokine_in_train"]))
    )

cytokine_cols = [f"{d}_present" for d in all_cytokines_in_train] + [f"{num_cytokines}_present" for num_cytokines in df["num_cytokines_in_train"].unique()]


In [8]:
df.head()

,mean_decoded_r_sq_per_cell_type,mean_e_distance_per_cell_type,mean_mmd_per_cell_type,mean_sdiv_10_per_cell_type,mean_sdiv_100_per_cell_type,mean_deg_r_sq_per_cell_type,mean_deg_e_distance_per_cell_type,mean_deg_mmd_per_cell_type,mean_deg_sdiv_10_per_cell_type,mean_deg_sdiv_100_per_cell_type,...,e_distance_pDC,mmd_pDC,div_10_pDC,div_100_pDC,deg_decoded_r_squared_pDC,deg_e_distance_pDC,deg_mmd_pDC,deg_div_10_pDC,deg_div_100_pDC,new_run
Donor8_IL-1Ra,0.936079,18.672976,0.010558,99.468785,15.969938,0.835870,27.137565,0.039739,137.779715,22.365724,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor12_IL-32-beta,0.880287,57.002564,0.019194,131.728360,43.893835,0.488720,78.095042,0.063817,178.566758,52.956970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor8_FasL,0.936261,20.548462,0.010757,101.143822,17.251914,0.816263,26.480086,0.042731,139.250231,22.540262,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor7_OX40L,0.923112,29.121615,0.011908,104.739377,22.172538,0.763669,36.172174,0.051545,146.038442,29.380676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
Donor5_BAFF,0.920968,30.491248,0.012784,105.387378,23.400425,0.689538,35.676550,0.049371,156.660900,28.123338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [9]:
cell_types = ["CD4 Naive", "B Naive", "CD8 Naive", "CD4 Memory", "Treg", "NKT", "CD8 Memory", "MAIT", "CD16 Mono", "CD14 Mono", "CD56-dim NK", "B Intermediate/Memory", "cDC"]

In [10]:
df_cs = {}
for ct in cell_types:
    target_variable = f"e_distance_{ct}"
    train_cytos_ordered = [f"{el}_present" for el in cyto_order if el in all_cytokines_in_train]
    test_cytos_ordered = [el for el in cyto_order if el in all_cytokines_in_test]
    df_c = pd.DataFrame(columns=cytokine_cols)
    for cytokine in all_cytokines_in_test:
        filtered_df = df[df["cytokine"]==cytokine]
        filtered_df = filtered_df[~filtered_df[target_variable].isnull()]
        lower_bound = filtered_df["ood_e_distance"].quantile(0.0)
        upper_bound = filtered_df["ood_e_distance"].quantile(0.95)
        filtered_df = filtered_df[(filtered_df["ood_e_distance"] >= lower_bound) & (filtered_df["ood_e_distance"] <= upper_bound)]
        d = get_cytokine_importance(all_cytokines_in_train, filtered_df, cytokine, target_variable=target_variable).set_index("Feature")
        d.loc[f"{cytokine}_present"] = None
        df_c.loc[cytokine] = d.loc[cytokine_cols]["Coefficient"]
    
    df_c = df_c.loc[test_cytos_ordered, train_cytos_ordered]
    df_cs[ct] = df_c
    ct_str = ct.replace(" ", "_").replace("/", "_")
    df_c.to_csv(os.path.join(data_output_dir, f"{ct_str}_feature_importance.csv"))
    
    